In [10]:
import pandas as pd
import numpy as np
us_total = pd.read_csv("us_total_vaccinations.csv")
us_total = us_total.drop(columns = ["daily_vaccinations_raw","daily_vaccinations_per_million", "people_vaccinated_per_hundred", "people_fully_vaccinated_per_hundred","distributed_per_hundred"], axis = 0)
us_total = us_total[pd.notnull(us_total["total_vaccinations"])]
us_total = us_total.fillna(0)
data = [us_total["date"], us_total["people_vaccinated"]]
headers = ["date", "people_vaccinated"]
us_total = pd.concat(data, axis=1, keys=headers)

us_total.head()

,date,people_vaccinated
0,12/20/20,0.0
1,12/21/20,0.0
2,12/23/20,0.0
3,12/26/20,0.0
4,12/28/20,0.0


In [2]:
import pandas as pd
import numpy as np

#unwanted_regions = ["American Samoa", "Bureau of Prisons", "Dept of Defense", "Federated States of Micronesia", "Guam", "Long Term Care", "Indian Health Svc", "Marshall Islands", "Northern Mariana Islands", "Puerto Rico", "Republic of Palau", "Virgin Islands", "Veterans Health", "United States"]
#clean_df = sys_df[sys_df['Entity'].isin(unwanted_regions) == False]
# I am thinking to first plot all 50 states for their percentage of people vaccinated over time using existing data only
# Once I have the trajectory prediction model working then we can redo this plot except with new data that reaches to August or September
#In order to do a national cumulative, we will need to multiply every entry by the respective state population then divide by 100.

print("Scenario One: reaching herd immunity assuming that all people who get vaccinated includes all the people that had Covid in the past")

us_pop = 332915073 #population value from macrotrends


negative_tweet_percent = .4

positive_tweet_percent = .3

neutral_tweet_percent = .3
while us_total.iloc[-1]['people_vaccinated'] < us_pop*.62: # 62% asap bucket
    ten_days = us_total.iloc[-9:]['people_vaccinated']

    ten_days_inc = ten_days.diff()

    ten_day_ave = np.mean(ten_days_inc)
    
    next_day_val = us_total.iloc[-1]['people_vaccinated'] + ten_day_ave

    next_day_date = (pd.to_datetime(us_total.iloc[-1]['date']) + pd.to_timedelta(np.ceil(1), unit="D")).date()

    us_total = us_total.append({'date': next_day_date, 'people_vaccinated': next_day_val}, ignore_index=True)



us_total['percent_vaccinated'] = us_total['people_vaccinated'] / us_pop



us_total.tail()
#Possibly add CI with smallest and largest increments in future scenarios"

Scenario One: reaching herd immunity assuming that all people who get vaccinated includes all the people that had Covid in the past


,date,people_vaccinated,percent_vaccinated
159,2021-06-11,2.020920e+08,0.607038
160,2021-06-12,2.034145e+08,0.611010
161,2021-06-13,2.047371e+08,0.614983
162,2021-06-14,2.060597e+08,0.618956
163,2021-06-15,2.073823e+08,0.622928


In [3]:
month_avg = (np.mean(us_total.iloc[-28:]['people_vaccinated'].diff()))*0.5864 #.5864 + 0.005 (2% over 4 months --> 1/2% each month)
while us_total.iloc[-1]['people_vaccinated'] < us_pop*.79: 
    #ten_days = nt_df.iloc[-9:]['people_vaccinated']

    #ten_days_inc = ten_days.diff()

    #ten_day_ave = np.mean(ten_days_inc) # multiply by slow in growth --> 12.5% each month to 7.33%
    #print(ten_day_ave)
    next_day_val = us_total.iloc[-1]['people_vaccinated'] + month_avg
    #print(pd.to_datetime(nt_df.iloc[-1]['Day']))

    next_day_date = (pd.to_datetime(us_total.iloc[-1]['date']) + pd.to_timedelta(np.ceil(1), unit="D")).date()

    us_total = us_total.append({'date': next_day_date, 'people_vaccinated': next_day_val}, ignore_index=True)
us_total['percent_vaccinated'] = us_total['people_vaccinated'] / us_pop
us_total.tail()

,date,people_vaccinated,percent_vaccinated
231,2021-08-22,2.601205e+08,0.781342
232,2021-08-23,2.608961e+08,0.783672
233,2021-08-24,2.616717e+08,0.786001
234,2021-08-25,2.624472e+08,0.788331
235,2021-08-26,2.632228e+08,0.790660


In [4]:
print("Looking at scenario one's dataset predictions, we can see that the country reaches 100% herd immunity on August 1st, 2021")

Looking at scenario one's dataset predictions, we can see that the country reaches 100% herd immunity on August 1st, 2021


In [21]:
us_total = pd.read_csv("us_total_vaccinations.csv")
us_total = us_total.drop(columns = ["daily_vaccinations_raw","daily_vaccinations_per_million", "people_vaccinated_per_hundred", "people_fully_vaccinated_per_hundred","distributed_per_hundred"], axis = 0)
us_total = us_total[pd.notnull(us_total["total_vaccinations"])]
us_total = us_total.fillna(0)
data = [us_total["date"], us_total["people_vaccinated"]]
headers = ["date", "people_vaccinated"]
us_total = pd.concat(data, axis=1, keys=headers)

us_total.tail()

,date,people_vaccinated
107,4/20/21,133266995.0
108,4/21/21,134445595.0
109,4/22/21,135791031.0
110,4/23/21,137234889.0
111,4/24/21,138644724.0


In [23]:
print("Scenario Two: reaching herd immunity assuming that 20% of people vaccinated had Covid")

us_pop = 332915073 #population value from macrotrends

covid_cases = pd.read_csv('United_States_COVID-19_Cases_and_Deaths_by_State.csv')

total_cases = covid_cases.groupby('submission_date').sum()
total_cases = total_cases.reset_index()
total_cases['submission_date'] = pd.to_datetime(total_cases['submission_date'])
total_cases = total_cases.sort_values(by="submission_date")

start_date = '2020-12-20'
end_date = '2021-04-24'
mask = (total_cases['submission_date'] >= start_date) & (total_cases['submission_date'] <= end_date)
total_cases_df = total_cases[(total_cases['submission_date'] >= start_date) & (total_cases['submission_date'] <= end_date)]

total_cases_df = total_cases_df.rename(columns={"submission_date": "Day"})

total_cases_df.tail()

total_vax = us_total

total_vax = total_vax.rename(columns={"date": "Day"})

total_vax['Day'] = pd.to_datetime(total_vax['Day'])

combined_df = total_vax.merge(total_cases_df, on='Day')

combined_df = combined_df.sort_values(by="Day")

combined_df['adjusted_vax'] = combined_df['people_vaccinated'] + .2 * combined_df['tot_cases']

combined_df = combined_df[['Day','adjusted_vax']]

combined_df.tail()

#grab sentiment analysis data on April 28th and plug it in here!!!

negative_tweet_percent = .4

positive_tweet_percent = .3

neutral_tweet_percent = .3

while combined_df.iloc[-1]['adjusted_vax'] < us_pop: 
    ten_days = combined_df.iloc[-9:]['adjusted_vax']

    ten_days_inc = ten_days.diff()

    ten_day_ave = np.mean(ten_days_inc)
    
    if (negative_tweet_percent > positive_tweet_percent) and (pd.to_datetime(combined_df.iloc[-1]['Day']) < pd.to_datetime('2021-05-28')): 
        ten_day_ave = ten_day_ave * 0.98

    next_day_val = combined_df.iloc[-1]['adjusted_vax'] + ten_day_ave

    next_day_date = (pd.to_datetime(combined_df.iloc[-1]['Day']) + pd.to_timedelta(np.ceil(1), unit="D")).date()
        
    combined_df = combined_df.append({'Day': next_day_date, 'adjusted_vax': next_day_val}, ignore_index=True)

combined_df['percent_vaccinated'] = combined_df['adjusted_vax'] / us_pop

combined_df.tail()



Scenario Two: reaching herd immunity assuming that 20% of people vaccinated had Covid


,Day,adjusted_vax,percent_vaccinated
269,2021-09-29,3.286862e+08,0.987297
270,2021-09-30,3.298325e+08,0.990741
271,2021-10-01,3.309788e+08,0.994184
272,2021-10-02,3.321251e+08,0.997627
273,2021-10-03,3.332715e+08,1.001070


In [7]:
print("Looking at scenario two's dataset predictions, we can see that the country reaches 100% herd immunity on July 28th, 2021")

Looking at scenario two's dataset predictions, we can see that the country reaches 100% herd immunity on July 28th, 2021
